但是虽说是 KMP，但是用到的只是求 next 的那一块，剩下的，得靠推公式...

我直接先来说下公式：


len(s) % (len(s) -  maxLen) = 0
其中 len(s) 为字符串 s 的长度，maxLen 为最长公共前后缀的长度。

所以这个公式翻译一下就是：如果 s 是周期串，那【s 的长度】是【s 的长度减去最长公共前后缀的长度】的倍数，那字符串 s 就是周期串。

作者：rocky0429-2
链接：https://leetcode.cn/problems/repeated-substring-pattern/solution/acm-xuan-shou-tu-jie-leetcode-zhong-fu-d-vl7i/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

In [ ]:
# https://writings.sh/post/algorithm-repeated-string-pattern

In [ ]:
class Solution:

    def repeatedSubstringPattern(self, s: str) -> bool:
        # 如果s由m个t重复形成 那么s的最长前后缀长度是 sn - tn == (m - 1) * tn
        # -> sn = m * tn -> 也就是 sn是tn的倍数
        n = len(s)
        nxt = [0] * n  # [0,i] 最长前后缀 前缀的结尾索引
        nxt[0] = -1
        j = -1
        for i in range(1, n):
            while j >= 0 and s[i] != s[j + 1]:
                j = nxt[j]
            if s[i] == s[j + 1]:
                j += 1
            nxt[i] = j
        mx = nxt[-1] + 1
        return mx != 0 and n % (n - mx) == 0

In [1]:
class Solution:

    def repeatedSubstringPattern(self, s: str) -> bool:
        next = [0] * len(s)
        now = 0
        i = 1
        while i < len(s):
            if s[i] == s[now]:
                next[i] = now + 1
                now += 1
                i += 1
            elif now:
                now = next[now - 1]
            else:
                i += 1
        return len(s) % (len(s) - now) == 0 and now != 0


In [1]:
class Solution:

    def repeatedSubstringPattern(self, s: str) -> bool:

        def kmp(s, p):
            next = [0] * len(p)
            now, i = 0, 1
            while i < len(p):
                if p[i] == p[now]:
                    next[i] = now + 1
                    now += 1
                    i += 1
                elif now:
                    now = next[now - 1]
                else:
                    next[i] = 0
                    i += 1
            i = 1
            j = 0  # i:匹配串 j:模式串
            while i < len(s) - 1:
                if s[i] == p[j]:
                    i += 1
                    j += 1
                elif j:
                    j = next[j - 1]
                else:
                    i += 1
                if j == len(p):
                    return True
            return False

        return kmp(s + s, s)


# 假设母串S是由子串s重复N次而成， 则 S+S则有子串s重复2N次， 那么现在有： S=Ns， S+S=2Ns， 其中N>=2。 如果条件成立， S+S=2Ns, 掐头去尾破坏2个s，S+S中还包含2*（N-1）s, 又因为N>=2, 因此S在(S+S)[1:-1]中必出现一次以上

In [4]:
# 超时
class Solution:

    def RabinKarp(self, s, p):  # p是否是s的字串 题目要求小写字母 因此26位即可
        getNum = lambda x: ord(x) - ord('a') # 不减ord('a)基准 也超时
        target = 0
        for n in p:
            target = target * 26 + getNum(n)
        num = 0
        l = 0
        lgh = len(p)
        for r in range(len(s)):
            num = num * 26 + getNum(s[r])
            while r - l + 1 > lgh:
                num -= getNum(s[l]) * (26**lgh)
                l += 1
            if r - l + 1 == lgh and num == target:
                return True
        return False

    def repeatedSubstringPattern(self, s: str) -> bool:
        return self.RabinKarp((s + s)[1:-1], s)


s = Solution()
s.repeatedSubstringPattern('abcabcabcabc')

bcabcabcabcabcabcabcab abcabcabcabc


True

In [11]:
# Rabin-Karp算法
# 不超时 这样防止溢出 同时减小内存占用 因为python整形长度靠内存
# 632ms


class Solution:

    def quickPower(self, num, k): # 没什么提速
        if k < 0:
            num = 1 / num
            k *= -1
        ans = 1
        while k:
            if k & 1 == 1:
                ans *= num
            num *= num
            k >>= 1
        return ans

    def strStr(self, haystack: str, needle: str) -> int:
        target = 0
        q = 1658598167  # 比较大的质数
        for n in needle:
            target = (256 * target % q + ord(n) % q) % q
        l = 0
        num = 0
        lgh = len(needle)
        rl = self.quickPower(256, lgh) % q
        for r in range(len(haystack)):
            num = (256 * num % q + ord(haystack[r]) % q) % q
            while r - l + 1 > lgh:
                num = (num % q - ord(haystack[l]) * rl % q + q) % q
                l += 1
            if r - l + 1 == lgh and num == target:
                if haystack[l:r + 1] == needle:
                    return True
        return False

    def repeatedSubstringPattern(self, s: str) -> bool:
        return self.strStr((s + s)[1:-1], s)


s = Solution()
s.repeatedSubstringPattern('abcabcabcabc')

True

In [10]:
def quickPower(num, k):
    if k < 0:
        num = 1 / num
        k *= -1
    ans = 1
    while k:
        if k & 1 == 1:
            ans *= num
        num *= num
        k >>= 1
    return ans


quickPower(51, 6), 51**6


(17596287801, 17596287801)